In [3]:
# pip install langchain -U #-i https://pypi.douban.com/simple

In [5]:
# pip install pydantic==1.10.8

In [8]:
# pip install sentence-transformers # -i https://pypi.douban.com/simple

In [10]:
# pip install chromadb

# RAG

## Internet

https://zhuanlan.zhihu.com/p/643233392

### 1. Get the Doc

In [11]:
import requests
from bs4 import BeautifulSoup

# url = "https://en.wikipedia.org/wiki/GPT-4"
url = "https://openai.com/research/gpt-4"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

# find the content div
content_div = soup.find('div', {'class': 'ui-block--text'})

# remove unwanted elements from div
unwanted_tags = ['sup', 'span', 'table', 'ul', 'ol']
for tag in unwanted_tags:
    for match in content_div.findAll(tag):
        match.extract()

print(content_div.get_text())

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

### 2. Split the Doc

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

article_text = content_div.get_text()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
)

texts = text_splitter.create_documents([article_text])
print(texts[0])
print(texts[1])

page_content='We’ve created GPT-4, the latest milestone in OpenAI’s effort in scaling up deep learning. GPT-4 is a'
page_content='GPT-4 is a large multimodal model (accepting image and text inputs, emitting text outputs) that,'


### 3. Text Chunks to Embeddings

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name='shibing624/text2vec-base-chinese')

In [10]:
# vector_store = FAISS.load_local(vs_path, self.embeddings)

from langchain.vectorstores import Chroma

# use the text chunks and the embeddings model to fill our vector store
db = Chroma.from_documents(texts, embedding)

In [3]:
from langchain import PromptTemplate

user_question = "ChatGPT and GPT4, which on is more stable?"

In [ ]:
# use our vector store to find similar text chunks
results = db.similarity_search(
    query=user_question,
    n_results=5
)

In [5]:
results = """
[Document(page_content='As a result, our GPT-4 training run was (for us at least!) unprecedentedly stable, becoming our'),
 Document(page_content='GPT-4’s text input capability via ChatGPT and the API (with a waitlist). To prepare the image input'),
 Document(page_content='We’ve created GPT-4, the latest milestone in OpenAI’s effort in scaling up deep learning. GPT-4 is a'),
 Document(page_content='GPT-4 is a large multimodal model (accepting image and text inputs, emitting text outputs) that,')]
"""

In [9]:
results

"\n[Document(page_content='As a result, our GPT-4 training run was (for us at least!) unprecedentedly stable, becoming our'),\n Document(page_content='GPT-4’s text input capability via ChatGPT and the API (with a waitlist). To prepare the image input'),\n Document(page_content='We’ve created GPT-4, the latest milestone in OpenAI’s effort in scaling up deep learning. GPT-4 is a'),\n Document(page_content='GPT-4 is a large multimodal model (accepting image and text inputs, emitting text outputs) that,')]\n"

In [10]:
# define the prompt template
template = """
You are a chat bot who loves to help people! Given the following context sections, answer the
question using only the given context. If you are unsure and the answer is not
explicitly writting in the documentation, say "Sorry, I don't know how to help with that."

Context sections:
{context}

Question:
{users_question}

Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "users_question"])

# fill the prompt template
prompt_text = prompt.format(context=results, users_question=user_question)

In [16]:
results

[Document(page_content='As a result, our GPT-4 training run was (for us at least!) unprecedentedly stable, becoming our'),
 Document(page_content='GPT-4’s text input capability via ChatGPT and the API (with a waitlist). To prepare the image input'),
 Document(page_content='We’ve created GPT-4, the latest milestone in OpenAI’s effort in scaling up deep learning. GPT-4 is a'),
 Document(page_content='GPT-4 is a large multimodal model (accepting image and text inputs, emitting text outputs) that,')]

### 4. Go to LLM

In [7]:
from load_model import *

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
# ask the defined LLM
llm(prompt_text)

'GPT-4 is more stable as per the given context.'

In [12]:
llm(user_question)

'It is difficult to determine which version of GPT (GPT3 or GPT4) is more stable as it depends on the specific context. However, GPT4 has been released recently and has some new features and improvements over GPT3. It is also designed to run more efficiently on hardware, which could make it more stable.'